<a href="https://colab.research.google.com/github/Winzen/mides-rascunho/blob/main/code/scraping/se/%5Bse%5Dextrair_mapeamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Verificar IP

In [ ]:
import requests

my_country = requests.get("https://api.myip.com/")
my_country.json()

{'ip': '34.16.136.187', 'country': 'United States', 'cc': 'US'}

# Importação

In [ ]:
import requests
import os
import csv
import shutil
import glob
import concurrent.futures
import datetime as dt
import time
import re
import pandas as pd
from time import sleep
from more_itertools import batched
from bs4 import BeautifulSoup
from urllib3 import PoolManager, disable_warnings, exceptions, Timeout
from urllib.parse import urlencode
from zipfile import ZipFile


class SlotBase:

  def __init__(self, municipio: str, ano: str, unidade: str,
               soup: BeautifulSoup , headers: dict, session: requests.Session) -> None:

    self.municipio = municipio
    self.ano = ano
    self.unidade = unidade
    self.soup = soup
    self.headers = headers
    self.session = session

def send_folder_drive(path_drive: str, path_input: str) -> None:

  if not len(os.listdir(path_input)) > 0:
    raise Exception("Essa Pasta vazia")
    return None

  shutil.make_archive(path_drive,
                      'zip',
                      path_input)


def send_solo_drive() -> None:

  drive.mount('/content/drive')

  send_folder_drive(path_drive_input + "_auto",
                    "/content/input/")

  send_folder_drive(path_drive_csv + "_auto",
                    "/content/csvs")


def ler(path: str) -> str:

  with open(path, "r") as slot_read:
    return slot_read.read()


def form_data_post(soup: BeautifulSoup, page: str = "Page$2") -> str|dict:

  inputs_temas = ["__EVENTTARGET", "__EVENTARGUMENT",
                    "__VIEWSTATE", "__VIEWSTATEGENERATOR",
                    "__SCROLLPOSITIONX", "__SCROLLPOSITIONY",
                    "__EVENTVALIDATION"]

  parametros= {key: soup.select_one(f"input#{key}").get("value") for key in inputs_temas}


  data = {
  "__EVENTTARGET": "ctl00$ContentPlaceHolder1$grdEmpenhos",
  "__EVENTARGUMENT": page,
  "__VIEWSTATE": parametros["__VIEWSTATE"],
  "__VIEWSTATEGENERATOR": parametros["__VIEWSTATEGENERATOR"],
  "__SCROLLPOSITIONX": parametros["__SCROLLPOSITIONX"],
  "__SCROLLPOSITIONY": parametros["__SCROLLPOSITIONY"],
  "__EVENTVALIDATION": parametros["__EVENTVALIDATION"],
  "ctl00$LoginView1$Login1$UserName": "",
  "ctl00$LoginView1$Login1$Password": "",
  "ctl00$ContentPlaceHolder1$hdfOrdenacao": "",
  }
  encoded_params = urlencode(data)
  # return encoded_params
  return encoded_params

def form_row_entidades(municipio: str, unidade: str, id_entidade: str, ano: str) -> dict:
  row = {
        "municipio": municipio,
        "entidade": unidade,
        "id_entidade": id_entidade,
        "ano": ano
          }
  return row

def make_list_chunks(values: list, chunk_size:int = 2000) -> list:
  resquet_in_chunks = [chunk for chunk in batched(values, chunk_size)]
  return resquet_in_chunks

def extrair(path, path_output) -> None:
  # loading the temp.zip and creating a zip object
  with ZipFile(path, 'r') as zObject:

      # Extracting all the members of the zip
      # into a specific location.
      zObject.extractall(
          path=path_output)


def get_anos_municipios() -> list:
  headers = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:125.0) Gecko/20100101 Firefox/125.0",
  "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
  "Accept-Language": "pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3",
  "Content-Type": "application/x-www-form-urlencoded",
  "Connection": "keep-alive"
  }

  link = "https://www.tcese.tc.br/portaldatransparencia/Default.aspx"

  with requests.Session() as s:

    response_anos_municipios = s.get(link, headers=headers)

  soup_anos_municipios = BeautifulSoup(response_anos_municipios.text)

  anos = [ano.get("value") for ano in soup_anos_municipios.select("select#ctl00_ContentPlaceHolder1_ddlAno option")]
  municipios = [municipio.get("value") for municipio in soup_anos_municipios.select("select#ctl00_ContentPlaceHolder1_ddlMunicipios option")]

  return [anos, municipios]

In [ ]:
def form_data_post(soup: BeautifulSoup, ano: str,
                   municipio: str, unidade: str, event_target: str = "") -> str|dict:

  inputs_temas = ["__EVENTTARGET", "__EVENTARGUMENT",
                  "__LASTFOCUS", "__VIEWSTATE", "__VIEWSTATEGENERATOR",
                  "__SCROLLPOSITIONX", "__SCROLLPOSITIONY",
                  "__EVENTVALIDATION"]

  parametros = {inp["name"]:inp.get("value") for inp in soup.select("input")}

  data = {
  "__EVENTTARGET": event_target,
  "__EVENTARGUMENT": "",
  "__LASTFOCUS": "",
  "__VIEWSTATE": parametros["__VIEWSTATE"],
  "__VIEWSTATEGENERATOR": parametros["__VIEWSTATEGENERATOR"],
  "__SCROLLPOSITIONX": parametros["__SCROLLPOSITIONX"],
  "__SCROLLPOSITIONY": parametros["__SCROLLPOSITIONY"],
  "__EVENTVALIDATION": parametros["__EVENTVALIDATION"],
  "ctl00$LoginView1$Login1$UserName": "",
  "ctl00$LoginView1$Login1$Password": "",
  "ctl00$ContentPlaceHolder1$ddlEsfera": "1",
  "ctl00$ContentPlaceHolder1$ddlAno": ano,
  "ctl00$ContentPlaceHolder1$ddlMunicipios": municipio,
  }

  encoded_params = urlencode(data)

  return encoded_params

In [ ]:
def form_data_post_ano(soup: BeautifulSoup, ano: str,
                   municipio: str, unidade: str, event_target: str = "") -> str|dict:

  inputs_temas = ["__EVENTTARGET", "__EVENTARGUMENT",
                  "__LASTFOCUS", "__VIEWSTATE", "__VIEWSTATEGENERATOR",
                  "__SCROLLPOSITIONX", "__SCROLLPOSITIONY",
                  "__EVENTVALIDATION"]

  parametros = {inp["name"]:inp.get("value") for inp in soup.select("input")}

  data = {
  "__EVENTTARGET": event_target,
  "__EVENTARGUMENT": "",
  "__LASTFOCUS": "",
  "__VIEWSTATE": parametros["__VIEWSTATE"],
  "__VIEWSTATEGENERATOR": parametros["__VIEWSTATEGENERATOR"],
  "__SCROLLPOSITIONX": parametros["__SCROLLPOSITIONX"],
  "__SCROLLPOSITIONY": parametros["__SCROLLPOSITIONY"],
  "__EVENTVALIDATION": parametros["__EVENTVALIDATION"],
  "ctl00$LoginView1$Login1$UserName": "",
  "ctl00$LoginView1$Login1$Password": "",
  "ctl00$ContentPlaceHolder1$ddlEsfera": "1",
  "ctl00$ContentPlaceHolder1$ddlAno": ano,
  }

  encoded_params = urlencode(data)
  # return encoded_params
  return encoded_params

In [ ]:
def form_data_post_municipio(soup: BeautifulSoup, ano: str,
                   municipio: str, unidade: str, event_target: str = "") -> str|dict:

  inputs_temas = ["__EVENTTARGET", "__EVENTARGUMENT",
                  "__LASTFOCUS", "__VIEWSTATE", "__VIEWSTATEGENERATOR",
                  "__SCROLLPOSITIONX", "__SCROLLPOSITIONY",
                  "__EVENTVALIDATION"]

  parametros = {inp["name"]:inp.get("value") for inp in soup.select("input")}

  data = {
  "__EVENTTARGET": event_target,
  "__EVENTARGUMENT": "",
  "__LASTFOCUS": "",
  "__VIEWSTATE": parametros["__VIEWSTATE"],
  "__VIEWSTATEGENERATOR": parametros["__VIEWSTATEGENERATOR"],
  "__SCROLLPOSITIONX": parametros["__SCROLLPOSITIONX"],
  "__SCROLLPOSITIONY": parametros["__SCROLLPOSITIONY"],
  "__EVENTVALIDATION": parametros["__EVENTVALIDATION"],
  "ctl00$LoginView1$Login1$UserName": "",
  "ctl00$LoginView1$Login1$Password": "",
  "ctl00$ContentPlaceHolder1$ddlEsfera": "1",
  "ctl00$ContentPlaceHolder1$ddlMunicipios": municipio
  }

  encoded_params = urlencode(data)

  return encoded_params

In [ ]:
def form_data_post_unidade(soup: BeautifulSoup, ano: str,
                   municipio: str, unidade: str, event_target: str = "") -> str|dict:

  inputs_temas = ["__EVENTTARGET", "__EVENTARGUMENT",
                  "__LASTFOCUS", "__VIEWSTATE", "__VIEWSTATEGENERATOR",
                  "__SCROLLPOSITIONX", "__SCROLLPOSITIONY",
                  "__EVENTVALIDATION"]

  parametros = {inp["name"]:inp.get("value") for inp in soup.select("input")}

  data = {
  "__EVENTTARGET": event_target,
  "__EVENTARGUMENT": "",
  "__LASTFOCUS": "",
  "__VIEWSTATE": parametros["__VIEWSTATE"],
  "__VIEWSTATEGENERATOR": parametros["__VIEWSTATEGENERATOR"],
  "__SCROLLPOSITIONX": parametros["__SCROLLPOSITIONX"],
  "__SCROLLPOSITIONY": parametros["__SCROLLPOSITIONY"],
  "__EVENTVALIDATION": parametros["__EVENTVALIDATION"],
  "ctl00$LoginView1$Login1$UserName": "",
  "ctl00$LoginView1$Login1$Password": "",
  "ctl00$ContentPlaceHolder1$ddlEsfera": "1",
  "ctl00$ContentPlaceHolder1$ddlUnidadeGestora":	unidade,
  "ctl00$ContentPlaceHolder1$imgConsultar.x":	"101",
  "ctl00$ContentPlaceHolder1$imgConsultar.y":	"16"
  }

  encoded_params = urlencode(data)

  return encoded_params

# Meu drive pessoal - Variaveis Globais

In [ ]:
path_drive_input = f"/content/drive/MyDrive/DataBase/world_bank/se/input_html_empenhos_map"
path_drive_csv = f"/content/drive/MyDrive/DataBase/world_bank/se/csvs"
# local_path_csv = f"/content/links_coleta_grupo/links_coletados_pagamentos_grupo_{id_grupo}.csv"

# Extrair CSV

In [ ]:
extrair(path=path_drive_csv + ".zip",
        path_output="/content/csvs")

In [ ]:
extrair(path=path_drive_input + ".zip",
        path_output="/content/input")

### Extrair Auto-Save

In [ ]:
extrair(path=path_drive_csv + "_auto" + ".zip",
        path_output="/content/csvs")

In [ ]:
extrair(path=path_drive_input + "_auto" + ".zip",
        path_output="/content/input")

# Formar DataFrame das entidades

In [ ]:
link_confirmar_unidade = "https://www.tcese.tc.br/portaldatransparencia/Default.aspx"
link_acesse_inicio_dados = "https://www.tcese.tc.br/portaldatransparencia/DadosUnidade.aspx"


headers = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:125.0) Gecko/20100101 Firefox/125.0",
  "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
  "Accept-Language": "pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3",
  "Content-Type": "application/x-www-form-urlencoded",
  "Connection": "keep-alive"
}

rows = []

anos, municipios = get_anos_municipios()

for municipio in municipios:
  for ano in anos:
    with requests.Session() as s:

      inicial_response = s.get(link_confirmar_unidade, headers=headers)
      soup_inicial = BeautifulSoup(inicial_response.text)
      select_ano = form_data_post_ano(soup_inicial, ano, municipio, unidade, "ctl00$ContentPlaceHolder1$ddlAno")
      # body_inicial = form_data_post(soup_inicial, ano, municipio, unidade)

      response_ano = s.post(link_confirmar_unidade, headers=headers, data=select_ano)
      soup_ano = BeautifulSoup(response_ano.text)
      select_municipio = form_data_post_municipio(soup_ano, ano, municipio, unidade, "ctl00$ContentPlaceHolder1$ddlMunicipios")
      response_municipio = s.post(link_confirmar_unidade, headers=headers, data=select_municipio)
      # body_inicial = form_data_post(soup_inicial, ano, municipio, unidade)

      # response2 = s.post(link_acesse_inicio_dados, headers=headers)
      soup_municipio = BeautifulSoup(response_municipio.text)

      select_unidade = "select#ctl00_ContentPlaceHolder1_ddlUnidadeGestora option"
      element_unidades = soup_municipio.select(select_unidade)
      unidades = [form_row_entidades(municipio, unidade.text, unidade.get("value"), ano) for unidade in element_unidades]
      rows += unidades

df = pd.DataFrame(rows)
df.to_csv("entidades.csv", index=False)

# Extrair Paginas de coleta (Mapeamento)

In [ ]:
def consultar_unidade(session: requests.Session, headers: dict, ano: str,
                   municipio: str, unidade: str) -> BeautifulSoup:

  link_confirmar_unidade = "https://www.tcese.tc.br/portaldatransparencia/Default.aspx"
  link_acesse_inicio_dados = "https://www.tcese.tc.br/portaldatransparencia/DadosUnidade.aspx"

  inicial_response = session.get(link_confirmar_unidade, headers=headers)
  soup_inicial = BeautifulSoup(inicial_response.text)
  select_ano = form_data_post_ano(soup_inicial, ano, municipio, unidade, "ctl00$ContentPlaceHolder1$ddlAno")

  response_ano = session.post(link_confirmar_unidade, headers=headers, data=select_ano)
  soup_ano = BeautifulSoup(response_ano.text)
  select_municipio = form_data_post_municipio(soup_ano, ano, municipio, unidade, "ctl00$ContentPlaceHolder1$ddlMunicipios")
  response_municipio = session.post(link_confirmar_unidade, headers=headers, data=select_municipio)

  soup_municipio = BeautifulSoup(response_municipio.text)
  select_unidade = form_data_post_unidade(soup_municipio, ano, municipio, unidade)
  response_unidade = session.post(link_confirmar_unidade, headers=headers, data=select_unidade)

  soup = BeautifulSoup(response_unidade.text)
  return soup

In [ ]:
def form_data_empenho(soup: BeautifulSoup) -> str|dict:

  parametros = {inp["name"]:inp.get("value") for inp in soup.select("input")}
  data = {
    "__EVENTTARGET": "",
    "__EVENTARGUMENT": "",
    "__LASTFOCUS": "",
    "__VIEWSTATE": parametros["__VIEWSTATE"],
    "__VIEWSTATEGENERATOR": parametros["__VIEWSTATEGENERATOR"],
    "__SCROLLPOSITIONX": parametros["__SCROLLPOSITIONX"],
    "__SCROLLPOSITIONY": parametros["__SCROLLPOSITIONY"],
    "__EVENTVALIDATION": parametros["__EVENTVALIDATION"],
    "ctl00$LoginView1$Login1$UserName":	"",
    "ctl00$LoginView1$Login1$Password":	"",
    "ctl00$ContentPlaceHolder1$txtDataInicial":	"01/01",
    "ctl00$ContentPlaceHolder1$txtDataFinal":	"31/12",
    "ctl00$ContentPlaceHolder1$txtNumero":	"",
    "ctl00$ContentPlaceHolder1$txtValorMin":	"",
    "ctl00$ContentPlaceHolder1$txtValorMax":	"",
    "ctl00$ContentPlaceHolder1$ddlOrgao":	"",
    "ctl00$ContentPlaceHolder1$ddlUnidadeOrcamentaria":	"",
    "ctl00$ContentPlaceHolder1$txtCpfCnpj":	"",
    "ctl00$ContentPlaceHolder1$txtEspecDespesa":	"",
    "ctl00$ContentPlaceHolder1$ddlRegiao":	"",
    "ctl00$ContentPlaceHolder1$ddlFuncao":	"",
    "ctl00$ContentPlaceHolder1$ddlSubfuncao":	"",
    "ctl00$ContentPlaceHolder1$ddlPrograma":	"",
    "ctl00$ContentPlaceHolder1$ddlProjAtividade":	"",
    "ctl00$ContentPlaceHolder1$ddlCategEconomica":	"",
    "ctl00$ContentPlaceHolder1$ddlGrupoDespesa":	"",
    "ctl00$ContentPlaceHolder1$ddlModalidadeApp":	"",
    "ctl00$ContentPlaceHolder1$ddlElementoDespesa":	"",
    "ctl00$ContentPlaceHolder1$ddlFonteRecurso":	"",
    "ctl00$ContentPlaceHolder1$btnConsultar":	"Consultar"
  }

  encoded_params = urlencode(data)
  return encoded_params

In [ ]:
def form_data_page(soup: BeautifulSoup, page= "1")-> str|dict:

  parametros = {inp["name"]:inp.get("value") for inp in soup.select("input")}

  data = {
  "__EVENTTARGET": "ctl00$ContentPlaceHolder1$grdEmpenhos",
  "__EVENTARGUMENT": f"Page${page}",
  "__VIEWSTATE": parametros["__VIEWSTATE"],
  "__VIEWSTATEGENERATOR": parametros["__VIEWSTATEGENERATOR"],
  "__SCROLLPOSITIONX": parametros["__SCROLLPOSITIONX"],
  "__SCROLLPOSITIONY": parametros["__SCROLLPOSITIONY"],
  "__EVENTVALIDATION": parametros["__EVENTVALIDATION"],
  "ctl00$LoginView1$Login1$UserName": "",
  "ctl00$LoginView1$Login1$Password": "",
  "ctl00$ContentPlaceHolder1$hdfOrdenacao	": "",
  }

  encoded_params = urlencode(data)
  return encoded_params

In [ ]:
class SlotExtration:

  def __init__(self, base:SlotBase, response: requests.Response, page: int) -> None:

    self.base = base
    self.response = response
    self.page = page

In [ ]:
def mudar_pagina(s: requests.Session, soup: BeautifulSoup, headers: dict, page: int|str) -> requests.Response:

  link_acessa_empenhos = "https://www.tcese.tc.br/portaldatransparencia/EmpenhoLista.aspx?Ini=01/01&Fim=31/12&Num=&Min=&Max=&Org=&UO=&Reg=&Fun=&SubF=&Prg=&GD=&Mod=&PA=&CE=&ED=&SED=&FR=&CPFCNPJ=&EspD="

  data_next_page = form_data_page(soup, str(page))
  response_next_page = s.post(link_acessa_empenhos, headers=headers, data=data_next_page)
  return response_next_page

In [ ]:
def get_response(base: SlotBase, page: int) -> SlotExtration:

  response_next_page = mudar_pagina(base.session, base.soup, base.headers, page)
  slot_extration = SlotExtration(base, response_next_page, page)
  return slot_extration

In [ ]:
def save_html(slot: SlotExtration) -> SlotExtration|bool:

  try:

    html = slot.response.text

    if "LISTA DE EMPENHOS" in html:
      path_base = f"input/{slot.base.ano}/{slot.base.municipio}/{slot.base.unidade}/"

      os.makedirs(path_base, exist_ok=True)

      with open(path_base + f"{slot.page}.html", "w") as form_html:
        form_html.write(html)

      return slot

    return False

  except:

    return False

In [ ]:
def verifica_ultima_pagina(soup: BeautifulSoup) -> str|bool:

  ultima_disponivel = False
  last_page = soup.select('img[title="Última Página"]')
  paginas = re.findall(r"Page[$](\d+)", str(soup))

  if last_page:
    ultima_disponivel = "Last"
  elif paginas:
    ultima_disponivel = str(max([int(n) for n in list(dict.fromkeys(paginas))]))

  return ultima_disponivel

def get_last_page(s: requests.Session, soup: BeautifulSoup, headers: dict) -> int:

 response =  mudar_pagina(s, soup, headers, "Last")
 suop = BeautifulSoup(response.text)
 pagina_atual = int(suop.select("td span")[-1].text)
 return pagina_atual

def form_paginas(s: requests.Session, soup: BeautifulSoup, headers: dict) -> list|bool:

  paginas = False

  ultima_pagina = verifica_ultima_pagina(soup)
  if ultima_pagina:

    if ultima_pagina == "Last":
      ultima_pagina = get_last_page(s, soup, headers)

    paginas = list(range(2, int(ultima_pagina) + 1))

  return paginas

In [ ]:
def consultar_empenho(s: requests.Session, headers: dict) -> list:

  link_confirmar_empenho = "https://www.tcese.tc.br/portaldatransparencia/Empenho.aspx"

  consulta_empenho = s.get(link_confirmar_empenho, headers=headers)
  soup_consulta_empenho = BeautifulSoup(consulta_empenho.text)
  data_empenho = form_data_empenho(soup_consulta_empenho)
  response_empenho_inicial = s.post(link_confirmar_empenho, headers=headers, data=data_empenho)

  soup_empenho_inicial = BeautifulSoup(response_empenho_inicial.text)
  return [soup_empenho_inicial, response_empenho_inicial]

In [ ]:
def coletar_paginas_empenho(municipio: str, ano: str, unidade: str) -> bool:

  headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:125.0) Gecko/20100101 Firefox/125.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3",
    "Content-Type": "application/x-www-form-urlencoded",
    "Connection": "keep-alive"
  }

  print(f"{'-' * 30}\nIniciando extração\nMunicipio: {municipio}\nUnidade: {unidade}\nAno: {ano}")

  start_time = time.time()

  with requests.Session() as s:

    consulta = consultar_unidade(s, headers, ano, municipio, unidade)
    soup_empenho_inicial, response_empenho_inicial = consultar_empenho(s, headers)

    if "LISTA DE EMPENHOS" in str(soup_empenho_inicial):

      slot_base = SlotBase(municipio, ano, unidade, soup_empenho_inicial, headers, s)

      first_page = SlotExtration(slot_base, response_empenho_inicial, 1)

      tasks_extration = [first_page]

      paginas = form_paginas(s, soup_empenho_inicial, headers)

      if paginas:

        if len(paginas) >= 11:

          paginas_chunk = make_list_chunks(paginas, chunk_size=10)

          for chuck in paginas_chunk:

            with concurrent.futures.ThreadPoolExecutor() as executor:
              tasks_extration += [response for response in executor.map(lambda page: get_response(slot_base, page), chuck)]

            slot_base.soup = BeautifulSoup(tasks_extration[-1].response.text)

        else:

          with concurrent.futures.ThreadPoolExecutor() as executor:
            tasks_extration += [response for response in executor.map(lambda page: get_response(slot_base, page), paginas)]

      print(f"Dados prontos para serem salvos: {len(tasks_extration)}")

      with concurrent.futures.ThreadPoolExecutor() as executor:
          extrations = [extration for extration in executor.map(save_html, tasks_extration) if extration]

      print(f"Dados salvos com sucesso: {len(extrations)}")

      segundo_execucao = time.time() - start_time
      print(f"Duração da execução: {segundo_execucao / 60:.2f}\n{'-' * 30}")
      print(f"Intervalo\nRespeito de {segundo_execucao: 0.2f} segudos ao servidor")
      sleep(segundo_execucao * 0.2)

      if len(tasks_extration) == len(extrations):
        print(f"Extração Perfeita!\n{'-' * 30}")
        return True
    else:
      print("Erro na primeira pagina. Nada é possivel!")

  return False



In [ ]:
df_entidades = pd.read_csv("/content/csvs/se_entidades_coleta.csv", dtype=str)
para_coletar = df_entidades[df_entidades["coletado"] == "False"]

for n, row in enumerate(para_coletar.itertuples()):

  coletado = coletar_paginas_empenho(row.municipio, row.ano, row.id_entidade)

  if coletado:
    df_entidades.loc[row.Index, "coletado"] = "True"

  if (n + 1) % 20 == 0:
    print("Salvando..")
    df_entidades.to_csv("/content/csvs/se_entidades_coleta.csv", index=False)
    send_solo_drive()
    print("Salvamento concluido.")


print("Salvando..")
df_entidades.to_csv("/content/csvs/se_entidades_coleta.csv", index=False)
send_solo_drive()
print("Salvamento concluido.")

Salvando..
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Salvamento concluido.


# Mandar no drive

In [ ]:
drive.mount('/content/drive')

df_entidades.to_csv("/content/csvs/se_entidades_coleta.csv", index=False)

send_folder_drive(path_drive_input,
                "/content/input/")

send_folder_drive(path_drive_csv,
                "/content/csvs")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
